# Graph-SLAM types and helper methods

This tuto introduces you to the variable types that we defined for our SLAM problem, and also to some methods that we pre-programmed for you.

## Overview

Our SLAM system consists of a single monocular camera, and some markers or tags from the AprilTag family, which we will consider as our landmarks.

Geometrically, each keyframe in the graph will be a 3D pose.

Also, each landmark in the graph will be a 3D pose.

Factors in the graph will be of three different types:

- Motion factor, from one keyframe to the next.
- Landmark factor, from one keyframe to one landmark.
- Prior factor, from one keyframe to some absolute value, or from one landmark to some absolute value.

Each factor will be associated with a particular cost function.

The total cost to be minimized by the solver will be the sum of the costs of each one of the factors.

## Variable types

### The class `OptiVariablePose3`

This is a Pose3 object that can be used in Casadi for symbolic computations. Our keyframes and landmarks will be of this type.

the class basically contains:
- an ID
- a position : a symbolic 3-vector 
- an orientation : a symbolic 3-vector
- a method to recover the rotation matrix
- the initial value of position : a numeric 3-vector
- the initial value of orientation : a numeric 3-vector

This is the prototype and constructor:


In [ ]:
import numpy as np
import pinocchio as pin
import casadi
from pinocchio import casadi as cpin
from gslam_april_tools import *


In [ ]:
class OptiVariablePose3:
    def __init__(self, opti, id, position, anglevector):
        self.id             = id
        self.position       = opti.variable(3)
        self.anglevector    = opti.variable(3)
        self.R              = exp3(self.anglevector)
        opti.set_initial(self.position, position)
        opti.set_initial(self.anglevector, anglevector)


To create a keyframe, or a landmark, simply do:

In [ ]:
# some necesary variables
opti = casadi.Opti()
pos_example = np.array([1,2,3])
ori_example = np.array([4,5,6])

# create one keyframe!
kf_id = 1
keyframe = OptiVariablePose3(opti, kf_id, pos_example, ori_example)

# create one landmark!
lmk_id = 11
landmark = OptiVariablePose3(opti, lmk_id, pos_example, ori_example)

Keyframes and landmarks are stored in Dictionnaries, indexed by their ID:

In [ ]:
keyframes = dict()
landmarks = dict()

keyframes[kf_id]  = keyframe
landmarks[lmk_id] = landmark

So, having an ID, one can retrieve the KF or the lmk easily:

In [ ]:
lmk = landmarks[lmk_id]
kf  = keyframes[kf_id]

Adding more kfs (or lmks) is easy, just remember to change ID each time:

In [ ]:
# increment ID and create new keyframe:
kf_id += 1
keyframe = OptiVariablePose3(opti, kf_id, pos_example, ori_example)
keyframes[kf_id]  = keyframe

### The class `Factor`

Factors in our example are fairly simple. There are three types, but all can be stored in a single class:
- Prior factor: concerns a single KF or a single Lmk
- Motion factor: concerns two consecutive KFs
- Landmark factor: concerns one KF and one lmk

We define the class `Factor` as follows:

In [ ]:
class Factor:
    def __init__(self, type, id, i, j, meas, sqrt_info):
        self.type = type
        self.id = id
        self.i = i
        self.j = j
        self.meas = meas
        self.sqrt_info = sqrt_info

The fields of `Factor` are explained as follows:
- `type`: string with values in `{'prior','motion','landmark'}`
- `id`: integer
- `i`: the ID of the first concerned object
- `j`: the ID of the second concerned object
- `meas`: the measurement
- `sqrt_info`: the square-root of the information matrix

So to create a factor, you need to know each of these fields.

Factors are stored in a dictionary, indexed by their own ID.

#### Prior factor

It constrains one keyframe:



In [ ]:
factors = dict()

fac_id = 0
kf_id = 1
dummy = 0
meas = np.array([0,0,0,0,0,0])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors

factor = Factor('prior', fac_id, kf_id, dummy, meas, sqrt_info)

factors[fac_id] = factor


#### Motion factor

It relates two consecutive keyframes:

In [ ]:
kf1_id = 1
kf2_id = 2
meas = np.array([1,2,3,  3,2,1])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors
fac_id += 1

factor = Factor('motion', fac_id, kf1_id, kf2_id, meas, sqrt_info)

factors[fac_id] = factor



#### Landmark factor

It relates a keyframe and a landmark measured from that keyframe:

In [ ]:
fac_id += 1
kf_id = 1
lmk_id = lmk_id # this was already defined above
meas = np.array([1,2,3,  3,2,1])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors

factor = Factor('landmark', fac_id, kf_id, lmk_id, meas, sqrt_info)

factors[fac_id] = factor


Knowing what kfs or lmks are referred to by the factor is easy: check the `type` and recover the objects accordingly:

In [ ]:
if factor.type == 'prior':
    keyframe = keyframes[factor.i]
    # do stuff with this KF
elif factor.type == 'motion':
    keyframe1 = keyframes[factor.i]
    keyframe2 = keyframes[factor.j]
    # do stuff with these KFs
else:
    keyframe = keyframes[factor.i]
    landmark = landmarks[factor.j]
    # do stuff with these KF and lmk


We use this mechanism to compute the cost to minimize:

In [ ]:
# define dummy cost functions
def cost_constant_position (sqrt_info, keyframe1, keyframe2):
    return 10
def cost_landmark (measurement, sqrt_info, keyframe, landmark):
    return 20
def cost_keyframe_prior (measurement, sqrt_info, keyframe):
    return 30

# compute total cost
def computeTotalCost(factors, keyframes, landmarks):
    totalcost = 0
    for fid in factors:
        factor = factors[fid]
        i = factor.i
        j = factor.j
        measurement = factor.meas
        sqrt_info   = factor.sqrt_info
        if factor.type == "motion":
            totalcost += cost_constant_position (sqrt_info, keyframes[i], keyframes[j])
        elif factor.type == "landmark":
            totalcost += cost_landmark (measurement, sqrt_info, keyframes[i], landmarks[j])
        elif factor.type == "prior":
            totalcost += cost_keyframe_prior (measurement, sqrt_info, keyframes[i])
        else:
            print('Error in the factor type: type not known')
    return totalcost

totalcost = computeTotalCost(factors, keyframes, landmarks)

print('total cost = ', totalcost)

where the cost of each type of factor is computed by a dedicated function in the example above, they are defined as dummy functions). We see these cost functions below.

## Cost functions

The cost function is responsible for computing the cost of a given factor. Since a factor involves a measurement and some states, we begin by computing the expected value of the measurement, given the states:
$$
e = h(x_1, x_2)
$$
where $h()$ is a particular observation model, and $x_i$ are the states involved in the factor.

Then we compare our measurement with the expected value, obtaining what we call the prediction error $z$:
$$
z = y - e = y - h(x_1,x_2)
$$

this error $z$ is then weighted by the square root of the information matrix, $W$, and then taken the sauqred norm:
$$
c = || W \cdot z ||^2
$$

Overall, the cost we have computed for a given factor $k$ is the following:
$$
c_k = ||W_k(y_k-h_k(x_{i_k},x_{j_k}))||^2
$$
and the total cost for $K$ factors is
$$
c = \sum_{k=0}^K c_k = \sum_{k=0}^K ||W_k(y_k-h_k(x_{i_k},x_{j_k}))||^2
$$
This is the cost to be minimized by the solver.



### Landmark observation cost function

We can observe teh computation of one of such costs for our SLAM. We study the case of a landmark observation from a keyframe. We note the following facts:
1. the measurement $y$ is that obtained by the Apriltag detector, in the form of a translation vector and a rotation vector, stacked in a 6-vector.
1. one of the states $x_1$ is a particular keyframe, indexed by field `i` in the factor
1. the other state $x_2$ is a landmark, indexed by field `j` in the factor
1. the error $z$ is computed according to a chain of operations:
   1. the measurement is a 6-vector with translational and rotational parts
      $$
      y = \begin{bmatrix}y_T \\ y_w\end{bmatrix}
      $$
   1. the expected measurement $e$ is a 3D composition of keyframe and landmark states. It is therefore a 3d pose with translation $e_T$ and rotation matrix $e_R$
      $$
      e = \begin{bmatrix}e_T \\ e_R\end{bmatrix}
      $$
   1. the error is the "difference" of the predicted above with the measurement, composed of translational and rotational parts,
      $$
      z = \begin{bmatrix}z_T \\ z_w\end{bmatrix}
      $$
      1. for the translational part, the difference is a mere subtraction
         $$
         z_T = y_T - e_T
         $$
      1. for the rotational part, this "difference" is done in the SO3 manifold, with,
         $$z_w = \log(e_R^\top \cdot \exp(y_w))$$

It seems complicated. A glipmse at the code may clarify things up:

In [ ]:
# landmark observations
def cost_landmark(meas, sqrt_info, keyframe_i, landmark_j):
    # predict: landmark pose wrt KF pose
    ppred = keyframe_i.R.T @ (landmark_j.position - keyframe_i.position)
    Rpred = keyframe_i.R.T @ landmark_j.R

    # error: use manifold tools for the orientation part
    perr = meas[0:3] - ppred
    Rerr = Rpred.T @ exp3(meas[3:6])
    werr = log3(Rerr)

    err = casadi.vertcat(perr, werr)
 
    # apply weight and compute squared norm
    cost = casadi.sumsqr(sqrt_info @ err) 
    return cost


### Motion cost function: costant position model

Since we have no motion sensor, we assume the motion is null at each step. This is known as a constant-position motion model. What we do is:
- predict the motion: compose $x_1$ and $x_2$ so that you obtain a prediction that corresponds to the frame `keyframe2` with respect to `keyframe1`.
- compare this prediction with the null motion, that is, $y = 0_{6\times1}$

Since $y=0$, we can write the function prototype for the constant-position motion model with no measurement input. To allow for real motion, we have to provide an information matrix that is small.

Now, define a similar cost function for the motion, that is, between two keyframes.



In [ ]:
def cost_constant_position(sqrt_info, keyframe_i, keyframe_j):
    # write y9our code here and change the return line below
    return 0  

### Prior keyframe cost function

Now, write a cost function to apply a prior pose to one keyframe:

In [ ]:
def cost_keyframe_prior(meas, sqrt_info, keyframe_i):
    return 0

### Prior landmark cost function

Finally, write a cost function to apply a prior pose to a landmark:

In [ ]:
def cost_landmark_prior(meas, sqrt_info, landmark):
    return 0